## Nettoyage des données

In [1]:
import pandas as pd
import re
import datetime
import time

In [2]:
data_frame = None
%store -r data_frame

In [3]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32720 entries, 0 to 32719
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   APP_Libelle_etablissement           32719 non-null  object
 1   SIRET                               32720 non-null  object
 2   Adresse_2_UA                        32292 non-null  object
 3   Code_postal                         32720 non-null  object
 4   Libelle_commune                     32720 non-null  object
 5   Numero_inspection                   32720 non-null  object
 6   Date_inspection                     32720 non-null  object
 7   APP_Libelle_activite_etablissement  32720 non-null  object
 8   Synthese_eval_sanit                 32720 non-null  object
 9   Agrement                            8787 non-null   object
 10  geores                              31961 non-null  object
 11  filtre                              24086 non-null  ob

In [4]:
data_frame.describe()

,APP_Libelle_etablissement,SIRET,Adresse_2_UA,Code_postal,Libelle_commune,Numero_inspection,Date_inspection,APP_Libelle_activite_etablissement,Synthese_eval_sanit,Agrement,geores,filtre,ods_type_activite
count,32719,32720,32292,32720,32720,32720,32720,32720,32720,8787,31961,24086,32720
unique,25744,28832,27101,4989,8460,32720,299,149,4,6835,26914,94,7
top,ECOLE PRIMAIRE PUBLIQUE,39814347900069,LE BOURG,75015,Paris 15e Arrondissement,16793484,2023-04-04T02:00:00+02:00,Restaurant,Satisfaisant,72080002,"48.859, 2.347",Restaurant,Autres
freq,162,12,91,557,536,1,251,11653,17976,10,50,11653,24737


In [5]:
data_frame.columns

Index(['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Code_postal',
       'Libelle_commune', 'Numero_inspection', 'Date_inspection',
       'APP_Libelle_activite_etablissement', 'Synthese_eval_sanit', 'Agrement',
       'geores', 'filtre', 'ods_type_activite'],
      dtype='object')

In [6]:
# Droper les colonnes qui ne sont pas :  Date_inspection, APP_Libelle_activite, geores, Synthese_eval_sanitaire
data_frame = data_frame.drop(['APP_Libelle_etablissement', 'SIRET', 'Adresse_2_UA', 'Code_postal', 'Agrement', 'filtre', 'ods_type_activite', 'Numero_inspection', 'Libelle_commune'], axis=1)

In [7]:
data_frame.columns

Index(['Date_inspection', 'APP_Libelle_activite_etablissement',
       'Synthese_eval_sanit', 'geores'],
      dtype='object')

In [8]:
# On crée une fonction qui va nous permettre de vérifier le format de la date avec cet example 2023-04-25T02:00:00+02:00, si la ligne est pas bonne on drop la ligne sinon on retourne la date en timestamp
def check_date_format(date):
    if re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{2}:\d{2}$', date):
        return date
    else:
        return None

In [9]:
# On applique la fonction sur la colonne Date_inspection
data_frame['Date_inspection'] = data_frame['Date_inspection'].apply(check_date_format)

# Transforme ' - ' value en null sur la colonne APP_Libelle_activite_etablissement
data_frame['APP_Libelle_activite_etablissement'] = data_frame['APP_Libelle_activite_etablissement'].replace('_', None)

# On drop les lignes qui ont une valeur null ou NaN
data_frame = data_frame.dropna()

In [10]:
# On decompose les datas en 2 parties : les features et les targets
# Les features sont les colonnes qui nous permettent de predire la target
# Les targets sont les colonnes est la colonne Synthese_eval_sanit

X = data_frame.drop(['Synthese_eval_sanit'], axis=1)
Y = data_frame['Synthese_eval_sanit']

In [11]:
X

,Date_inspection,APP_Libelle_activite_etablissement,geores
0,2023-04-25T02:00:00+02:00,Restaurant,"49.985274, 1.561386"
1,2023-02-23T01:00:00+01:00,Restaurant,"46.15861, 6.677963"
2,2022-10-04T02:00:00+02:00,Alimentation générale,"48.881317, 2.23746"
4,2023-03-08T01:00:00+01:00,Alimentation générale,"48.844091, 2.219818"
5,2023-04-25T02:00:00+02:00,Alimentation générale,"48.072687, -4.185142"
...,...,...,...
32715,2023-03-30T02:00:00+02:00,Chaîne d'abattage de volaille/lagomorphe/petit...,"-21.206368, 55.337063"
32716,2022-11-24T01:00:00+01:00,Restaurant,"48.873997, 2.341843"
32717,2023-04-17T02:00:00+02:00,Transformation de produits carnés,"44.078462, 5.999322"
32718,2023-06-16T02:00:00+02:00,Restaurant,"45.738402, 4.837636"


In [12]:
Y

0             Satisfaisant
1             Satisfaisant
2             Satisfaisant
4              A améliorer
5        Très satisfaisant
               ...        
32715         Satisfaisant
32716         Satisfaisant
32717         Satisfaisant
32718         Satisfaisant
32719         Satisfaisant
Name: Synthese_eval_sanit, Length: 31922, dtype: object

In [13]:
# split colonne geores en 2 colonnes : latitude et longitude
X[['latitude', 'longitude']] = X['geores'].str.split(',', expand=True)
X = X.drop(['geores'], axis=1)

In [14]:
X

,Date_inspection,APP_Libelle_activite_etablissement,latitude,longitude
0,2023-04-25T02:00:00+02:00,Restaurant,49.985274,1.561386
1,2023-02-23T01:00:00+01:00,Restaurant,46.15861,6.677963
2,2022-10-04T02:00:00+02:00,Alimentation générale,48.881317,2.23746
4,2023-03-08T01:00:00+01:00,Alimentation générale,48.844091,2.219818
5,2023-04-25T02:00:00+02:00,Alimentation générale,48.072687,-4.185142
...,...,...,...,...
32715,2023-03-30T02:00:00+02:00,Chaîne d'abattage de volaille/lagomorphe/petit...,-21.206368,55.337063
32716,2022-11-24T01:00:00+01:00,Restaurant,48.873997,2.341843
32717,2023-04-17T02:00:00+02:00,Transformation de produits carnés,44.078462,5.999322
32718,2023-06-16T02:00:00+02:00,Restaurant,45.738402,4.837636


In [15]:
#On crée une fonction pour convertir la date d'inspection en timestamp
def date_to_timestamp(string):
    split = string.split('+')
    return time.mktime(datetime.datetime.strptime(split[0],"%Y-%m-%dT%H:%M:%S").timetuple())

In [16]:
#On applique la fonction à la colonne date
X['Date_inspection'] = X['Date_inspection'].apply(date_to_timestamp)
X

,Date_inspection,APP_Libelle_activite_etablissement,latitude,longitude
0,1.682381e+09,Restaurant,49.985274,1.561386
1,1.677110e+09,Restaurant,46.15861,6.677963
2,1.664842e+09,Alimentation générale,48.881317,2.23746
4,1.678234e+09,Alimentation générale,48.844091,2.219818
5,1.682381e+09,Alimentation générale,48.072687,-4.185142
...,...,...,...,...
32715,1.680134e+09,Chaîne d'abattage de volaille/lagomorphe/petit...,-21.206368,55.337063
32716,1.669248e+09,Restaurant,48.873997,2.341843
32717,1.681690e+09,Transformation de produits carnés,44.078462,5.999322
32718,1.686874e+09,Restaurant,45.738402,4.837636


In [17]:
# On créer une fonction qui supprime les caractères spéciaux et les espaces dans une chaine de caractère
def remove_special_characters_and_spaces(string):
    return re.sub(r'[^a-zA-Z0-9]', '', string)

In [18]:
# On applique la fonction sur toutes les colonnes
X['APP_Libelle_activite_etablissement'] = X['APP_Libelle_activite_etablissement'].apply(remove_special_characters_and_spaces)
X['latitude'] = X['latitude'].apply(remove_special_characters_and_spaces)
X['longitude'] = X['longitude'].apply(remove_special_characters_and_spaces)

In [19]:
# On transforme les colonnes latitude et longitude en float
X['latitude'] = X['latitude'].astype(float)
X['longitude'] = X['longitude'].astype(float)

In [20]:
X

,Date_inspection,APP_Libelle_activite_etablissement,latitude,longitude
0,1.682381e+09,Restaurant,49985274.0,1561386.0
1,1.677110e+09,Restaurant,4615861.0,6677963.0
2,1.664842e+09,Alimentationgnrale,48881317.0,223746.0
4,1.678234e+09,Alimentationgnrale,48844091.0,2219818.0
5,1.682381e+09,Alimentationgnrale,48072687.0,4185142.0
...,...,...,...,...
32715,1.680134e+09,Chanedabattagedevolaillelagomorphepetitgibier,21206368.0,55337063.0
32716,1.669248e+09,Restaurant,48873997.0,2341843.0
32717,1.681690e+09,Transformationdeproduitscarns,44078462.0,5999322.0
32718,1.686874e+09,Restaurant,45738402.0,4837636.0


In [21]:
# Formate les données avec le get_dummies
X['APP_Libelle_activite_etablissement'] = pd.factorize(X['APP_Libelle_activite_etablissement'])[0]

In [22]:
# stockage des données
%store X
%store Y

Stored 'X' (DataFrame)
Stored 'Y' (Series)
